# Deteksi Outlier dengan Local Outlier Factor (LOF)

## Apa itu LOF?

Dalam analisis deteksi anomali, Local Outlier Factor (LOF) adalah sebuah algoritma yang digunakan untuk mengidentifikasi data yang menyimpang dengan menilai seberapa jauh suatu titik data berbeda dari tetangga sekitarnya. Algoritma ini pertama kali diperkenalkan pada tahun 2000 oleh Markus M. Breunig, Hans-Peter Kriegel, Raymond T. Ng, dan Jörg Sander. LOF mengadopsi konsep serupa dengan DBSCAN dan OPTICS, yakni menggunakan "jarak inti" dan "jarak keterjangkauan" untuk memperkirakan kerapatan lokal suatu titik data.

## Tahapan LOF

1. **Menentukan parameter LOF** -- Langkah pertama dalam metode Local Outlier Factor (LOF) adalah menentukan parameter utama, yaitu nilai k dan metrik jarak yang akan digunakan. Nilai k menentukan jumlah tetangga terdekat yang digunakan untuk mengevaluasi kepadatan suatu titik data. Pemilihan k yang terlalu kecil dapat menyebabkan model terlalu sensitif terhadap pencilan lokal, sementara nilai yang terlalu besar dapat menyebabkan anomali sulit terdeteksi.

2. **Menentukan k distance dan tetangga terdekat** -- Setelah parameter ditentukan, langkah berikutnya adalah menghitung k-Distance untuk setiap titik dalam dataset. k-Distance adalah jarak dari suatu titik ke tetangga ke-k terdekatnya. Setelah nilai k-Distance diperoleh, daftar k-Nearest Neighbors (k-NN) ditentukan, yaitu sekumpulan titik yang berada dalam radius k-Distance tersebut. Titik-titik dalam k-NN akan menjadi acuan dalam perhitungan kepadatan lokal pada langkah-langkah selanjutnya.

3. **Menghitung Reachability Distance** -- Pada tahap ini, Reachability Distance dihitung untuk setiap pasangan titik yang berada dalam k-NN. Nilai ini ditentukan dengan mengambil nilai maksimum antara k-Distance dari tetangga dan jarak sebenarnya antara dua titik tersebut. Tujuan dari langkah ini adalah untuk memastikan bahwa titik yang sangat dekat dengan tetangganya tetap memiliki jarak keterjangkauan minimal setara dengan k-Distance dari tetangganya, sehingga distribusi kepadatan data lebih stabil.

4. **Menghitung Local Reachability Density (LRD)** -- Setelah Reachability Distance dihitung, langkah berikutnya adalah menentukan Local Reachability Density (LRD) untuk setiap titik. LRD mengukur kepadatan lokal dengan menghitung kebalikan dari rata-rata Reachability Distance antara titik tersebut dan tetangganya. Semakin kecil nilai LRD, semakin rendah kepadatan lokal suatu titik dibandingkan dengan tetangganya. Titik dengan kepadatan rendah dibandingkan lingkungannya berpotensi menjadi anomali.

5. **Menghitung Local Outlier Factor (LOF)** -- Setelah LRD setiap titik diketahui, nilai Local Outlier Factor (LOF) dihitung dengan membandingkan LRD suatu titik terhadap LRD dari tetangga-tetangganya. Nilai LOF diperoleh dengan menghitung rata-rata rasio LRD tetangga terhadap LRD titik tersebut. Jika LOF mendekati 1, artinya kepadatan titik tersebut sebanding dengan tetangganya dan tidak dianggap sebagai pencilan. Namun, jika LOF jauh lebih besar dari 1, titik tersebut memiliki kepadatan yang jauh lebih rendah dibandingkan dengan lingkungannya dan berpotensi menjadi anomali.

6. **Menentukan Anomali Berdasarkan LOF** -- Tahap terakhir adalah menetapkan ambang batas LOF untuk mengidentifikasi titik-titik yang dianggap sebagai anomali. Biasanya, titik dengan nilai LOF lebih besar dari 1.5 atau 2 dikategorikan sebagai pencilan, tergantung pada sensitivitas yang diinginkan. Dengan demikian, metode LOF dapat secara efektif mendeteksi anomali dalam dataset yang memiliki kepadatan tidak seragam, seperti klaster dengan ukuran berbeda atau data dengan distribusi yang kompleks.

## CONTOH MENGHITUNG MANUAL LOF

Berikut adalah cara menghitung **Local Outlier Factor (LOF)** untuk satu titik data dengan dua fitur:  

### 1. **Dataset**  

Misalkan kita memiliki dataset berikut:  

| ID | Feature1 | Feature2 |  
|----|----------|----------|  
| 1  | 1.0      | 2.0      |  
| 2  | 2.5      | 3.5      |  
| 3  | 3.0      | 4.0      |  
| 4  | 4.5      | 5.0      |  
| 5  | 5.0      | 6.0      |  
| 6  | 6.5      | 7.5      |  
| 7  | 7.0      | 8.0      |  
| 8  | 8.5      | 9.0      |  
| 9  | 9.0      | 10.0     |  
| 10 | 20.0     | 22.0     |  

Kita akan menghitung LOF untuk **titik 10** (20.0, 22.0).  

### 2. **Hitung Jarak Euclidean**  

Jarak Euclidean antara dua titik **(x1, y1)** dan **(x2, y2)** dihitung dengan rumus:

$
D = \sqrt{(x_2 - x_1)^2 + (y_2 - y_1)^2}
$

Menghitung jarak antara titik 10 dan semua titik lainnya:

- Titik 10 ke Titik 1:
  $
  D = \sqrt{(20.0 - 1.0)^2 + (22.0 - 2.0)^2}
    = \sqrt{361 + 400}
    = \sqrt{761}
    \approx 27.58
  $

- Titik 10 ke Titik 2:
  $
  D = \sqrt{(20.0 - 2.5)^2 + (22.0 - 3.5)^2}
    = \sqrt{306.25 + 342.25}
    = \sqrt{648.5}
    \approx 25.47
  $

- Titik 10 ke Titik 3:
  $
  D = \sqrt{(20.0 - 3.0)^2 + (22.0 - 4.0)^2}
    = \sqrt{289 + 324}
    = \sqrt{613}
    \approx 24.76
  $

- Titik 10 ke Titik 4:
  $
  D = \sqrt{(20.0 - 4.5)^2 + (22.0 - 5.0)^2}
    = \sqrt{240.25 + 289}
    = \sqrt{529.25}
    \approx 23.00
  $

- Titik 10 ke Titik 5:
  $
  D = \sqrt{(20.0 - 5.0)^2 + (22.0 - 6.0)^2}
    = \sqrt{225 + 256}
    = \sqrt{481}
    \approx 21.93
  $

- Titik 10 ke Titik 6:
  $
  D = \sqrt{(20.0 - 6.5)^2 + (22.0 - 7.5)^2}
    = \sqrt{182.25 + 210.25}
    = \sqrt{392.5}
    \approx 19.81
  $

- Titik 10 ke Titik 7:
  $
  D = \sqrt{(20.0 - 7.0)^2 + (22.0 - 8.0)^2}
    = \sqrt{169 + 196}
    = \sqrt{365}
    \approx 19.10
  $

- Titik 10 ke Titik 8:
  $
  D = \sqrt{(20.0 - 8.5)^2 + (22.0 - 9.0)^2}
    = \sqrt{132.25 + 169}
    = \sqrt{301.25}
    \approx 17.36
  $

- Titik 10 ke Titik 9:
  $
  D = \sqrt{(20.0 - 9.0)^2 + (22.0 - 10.0)^2}
    = \sqrt{121 + 144}
    = \sqrt{265}
    \approx 16.28
  $
### 3. **Tentukan k-Tetangga Terdekat** (k=2)  

Dua tetangga terdekat untuk titik 10 adalah **titik 9 dan titik 8**.  

### 4. **Hitung Reachability Distance**  

Reachability distance dihitung dengan rumus:

$
\text{reach-dist}(p, o) = \max(\text{k-distance}(o), \text{distance}(p, o))
$

Dengan asumsi bahwa **k-distance** untuk titik 9 dan titik 8 adalah jarak dari mereka ke tetangga terdekat kedua mereka:

- Misalkan **k-distance(9) = 1.5**
- Misalkan **k-distance(8) = 2.0**

Maka:

- $
  \text{reach-dist}(10,9) = \max(1.5, 16.28) = 16.28
  $
- $
  \text{reach-dist}(10,8) = \max(2.0, 17.36) = 17.36
  $

### 5. **Hitung Local Reachability Density (LRD)**  

$
\text{LRD}(10) = \frac{1}{\frac{1}{2} \times (16.28 + 17.36)}
$

$
= \frac{1}{\frac{1}{2} \times 33.64}
= \frac{1}{16.82}
\approx 0.06
$

### 6. **Hitung LOF**  

$
\text{LOF}(10) = \frac{0.06}{0.35} + \frac{0.06}{0.35} = 5.71
$

### 7. **Interpretasi**  

Karena **LOF = 5.71** (jauh lebih besar dari 1), titik 10 dianggap **outlier**.  

LOF menunjukkan bahwa titik ini sangat berbeda dibandingkan dengan tetangganya.



## Contoh Implementasi Menggunakan sklearn

In [6]:
%pip install pymysql
%pip install psycopg2

In [7]:
import psycopg2
import pymysql
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.neighbors import LocalOutlierFactor
from IPython.display import display, HTML

In [8]:
# Dataset baru dengan 10 baris data
data = {
    'Fitur 1': [1.0, 2.5, 3.0, 4.5, 5.0, 6.5, 7.0, 8.5, 9.0, 20.0],
    'Fitur 2': [2.0, 3.5, 4.0, 5.0, 6.0, 7.5, 8.0, 9.0, 10.0, 22.0]
}

# Membuat DataFrame
df = pd.DataFrame(data)

# Mengatur index mulai dari 1
df.index = np.arange(1, len(df) + 1)

# Inisialisasi model LOF dengan k=2 (2 tetangga terdekat)
lof = LocalOutlierFactor(n_neighbors=2)

# Fit model LOF dan prediksi label (1 untuk normal, -1 untuk outlier)
lof_labels = lof.fit_predict(df)

# Menambahkan hasil prediksi ke DataFrame
df['LOF Label'] = lof_labels

# Menampilkan hasil
display(HTML(df.to_html()))

,Fitur 1,Fitur 2,LOF Label
1,1.0,2.0,1
2,2.5,3.5,1
3,3.0,4.0,1
4,4.5,5.0,1
5,5.0,6.0,1
6,6.5,7.5,1
7,7.0,8.0,1
8,8.5,9.0,1
9,9.0,10.0,1
10,20.0,22.0,-1


In [9]:
# Menampilkan jumlah outlier
num_outliers = (lof_labels == -1).sum()
print(f"Jumlah Outlier: {num_outliers}")

Jumlah Outlier: 1
